In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install vadersentiment
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 12.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 6.8 MB/s 
     |████████████████████████████████| 662 kB 55.5 MB/s 
     |████████████████████████████████| 85 kB 5.3 MB/s 
     |████████████████████████████████| 5.2 MB 57.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 88 kB 6.2 MB/s 
     |████████████████████████████████| 5.8 MB 48.8 MB/s 
     |████████████████████████████████| 1.3 MB 61.7 MB/s 
     |████████████████████████████████| 182 kB 78.1 MB/s 
     |████████████████████████████████| 7.6 MB 48.5 MB/s 
     |████████████████████████████████| 1.1 MB 54.5 MB/s 
  Created wheel for hdbscan: filename=hdbsca

In [3]:
import pandas as pd
import numpy as np
import gensim
from nltk.stem import WordNetLemmatizer, SnowballStemmer
np.random.seed(10)
import nltk
from bertopic import BERTopic
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
import seaborn as sns
import plotly.express as px
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [12]:
import pandas as pd
file_name = "/content/drive/MyDrive/raw_reddit_politics_posts_comments.csv"
df = pd.read_csv(file_name)
df.head()

,id,subreddit,author,created_utc,domain,url,title,num_comments,selftext,score,comments
0,j767z3,politics,I_Mispelled_Mispell,2020-10-08 03:59:21,buzzfeednews.com,https://www.buzzfeednews.com/article/stephanie...,Kamala Harris's Energy Towards Pence's Interru...,29,NaN,1,I would bet the writer is white....What the fu...
1,j76abt,politics,Xeelee1123,2020-10-08 04:03:49,theguardian.com,https://www.theguardian.com/us-news/2020/oct/0...,Trump calls Covid diagnosis 'blessing from God...,29,NaN,1,Hi `Xeelee1123`. Thank you for participating i...
2,j76cwv,politics,slagholer,2020-10-08 04:08:46,talkingpointsmemo.com,https://talkingpointsmemo.com/news/pence-remin...,Pence Reminds Us All That The Pre-Trump GOP Is...,71,NaN,1,"But you guys, Trump has Jewish grandchildren. ..."
3,j76e7j,politics,NotsoPG,2020-10-08 04:11:19,democracynow.org,https://www.democracynow.org/2020/10/7/headlin...,"NYT: Jeff Sessions Defended Child Separations,...",14,NaN,1,He got better at picking the worst people over...
4,j76eku,politics,0belvedere,2020-10-08 04:12:05,nytimes.com,https://www.nytimes.com/2020/10/07/us/politics...,A fly sat atop Mike Pence's head for two minut...,44,NaN,1,well put. thanks....The fly explains its think...


In [13]:
rel_cols = list(df.columns)

In [14]:
file_name1 = "/content/drive/MyDrive/raw_reddit_politics_posts_comments_2.csv"
df2 = pd.read_csv(file_name1)
df2.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning:

Columns (7,9,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143) have mixed types.Specify dtype option on import or set low_memory=False.



,id,subreddit,author,created_utc,domain,url,title,num_comments,selftext,score,...,Unnamed: 134,Unnamed: 135,Unnamed: 136,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140,Unnamed: 141,Unnamed: 142,Unnamed: 143
0,jkek6x,politics,trifecta,29-10-2020 17:33,wfla.com,https://www.wfla.com/news/florida/florida-woma...,Florida woman in labor makes husband stop at e...,314,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,jkek7q,politics,freebandgang25,29-10-2020 17:33,cnn.com,https://www.cnn.com/2020/10/29/politics/florid...,Biden and Trump head to Florida for dueling ra...,7,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,jkel2l,politics,Mason0001,29-10-2020 17:35,bbc.com,https://www.bbc.com/news/amp/world-54553132,Hunter Biden: What was he doing in Ukraine and...,47,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,jkelpw,politics,freebandgang25,29-10-2020 17:35,cnn.com,https://www.cnn.com/2020/10/29/politics/f-35-j...,Top Democrat says Trump administration moving ...,18,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,jkelt9,politics,MadeInOne1,29-10-2020 17:36,commondreams.org,https://www.commondreams.org/views/2020/10/29/...,Let's Be Very Clear: Senate Republicans Have F...,308,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df3 = df2[rel_cols]

In [16]:
df3.shape

(74256, 11)

In [17]:
df_final = pd.concat([df,df3],ignore_index = True)

In [18]:
df_final.shape

(87656, 11)

In [17]:
#df_final.to_csv('/content/drive/MyDrive/raw_reddit_politics_posts_and_comments',index = False)

In [19]:
df = df_final.copy()

In [20]:
def clean_text(text):
    text = text.lower() # lowercase everything
    text = text.encode('ascii', 'ignore').decode()  # remove unicode characters
    text = re.sub(r'https*\S+', ' ', text) # remove links
    text = re.sub(r'http*\S+', ' ', text)
    # cleaning up text
    text = re.sub(r'\'\w+', '', text) 
    text = re.sub(r'\w*\d+\w*', '', text)
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(r'\s[^\w\s]\s', '', text)
    text = re.sub("[\(\[].*?[\)\]]", "", text)
    text = re.sub('[^a-zA-Z0-9 \.]', '', text)
    return text

In [21]:
def lemmatize(text):
  return WordNetLemmatizer().lemmatize(text, pos='v')
def preprocess(text):
    if type(text)!=str:
      return ''
    result = []
    text = text.replace('[removed]', '').replace('[deleted]', '')
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2:
            result.append(lemmatize(token))
    return ' '.join(result)

In [22]:
print(df.shape)
df.fillna("", inplace=True)
len(df[df.id==""])

(87656, 11)


338

In [23]:
df["title"] = df["title"].astype(str) 
df["selftext"] = df["selftext"].astype(str)
df["title"] = df["title"] + " " + df["selftext"]

In [69]:
df['cleaned_title'] = df.title.apply(preprocess)
df = df.dropna()
titles = df["cleaned_title"].astype(str).to_list()

In [ ]:
len(titles)

In [71]:
topic_model = BERTopic(min_topic_size=30, n_gram_range=(1,3), nr_topics='auto', verbose=True)
topics, _ = topic_model.fit_transform(titles)

Batches:   0%|          | 0/2722 [00:00<?, ?it/s]

2022-12-02 15:44:38,644 - BERTopic - Transformed documents to Embeddings
2022-12-02 15:46:40,153 - BERTopic - Reduced dimensionality
2022-12-02 15:46:50,782 - BERTopic - Clustered reduced embeddings
2022-12-02 15:47:09,577 - BERTopic - Reduced number of topics from 509 to 388


In [72]:
topic_model.save("/content/drive/MyDrive/topic_politics_model")

/usr/local/lib/python3.8/dist-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [4]:
# topic_model = BERTopic.load("/content/drive/MyDrive/topic_politics_model")

In [84]:
freq = topic_model.visualize_barchart(top_n_topics=10, n_words=10, height=400)
freq

In [85]:
freq.write_html("/content/drive/MyDrive/politics_word_freq.html")

In [74]:
intertopic_dist = topic_model.visualize_topics()
intertopic_dist

In [75]:
intertopic_dist.write_html("/content/drive/MyDrive/politics_intertopic_distance.html")

In [98]:
freq = topic_model.get_topic_info()
freq.head(10)

,Topic,Count,Name
0,-1,32700,-1_trump_biden_capitol_election
1,0,1823,0_covid_coronavirus_deaths_case
2,1,1608,1_court_supreme court_supreme_pennsylvania
3,2,1249,2_poll_joe biden_joe_biden
4,3,1107,3_capitol_riot_capitol riot_rioters
5,4,1023,4_michigan_certify_vote certify_wayne
6,5,952,5_giuliani_test_positive_test positive
7,6,738,6_barrett_coney barrett_coney_amy coney barrett
8,7,734,7_pelosi_nancy_nancy pelosi_pelosi office
9,8,725,8_iran_nuclear_israel_nuclear deal


In [99]:
topic_nr = freq.iloc[3]["Topic"]  # We select a frequent topic
topic_model.get_topic(topic_nr)

[('poll', 0.020120195966791123),
 ('joe biden', 0.01574087869348848),
 ('joe', 0.01545398482599349),
 ('biden', 0.012078843006795641),
 ('biden lead', 0.009756419444775209),
 ('lead', 0.008975929501714417),
 ('poll biden', 0.008380594385439553),
 ('lead trump', 0.007593871104284645),
 ('joe biden win', 0.006540389719464178),
 ('biden win', 0.006337373391162106)]

In [100]:
freq["word_strength"] = freq.Topic.apply(topic_model.get_topic)
freq.head()

,Topic,Count,Name,word_strength
0,-1,32700,-1_trump_biden_capitol_election,"[(trump, 0.0025589781851013994), (biden, 0.002..."
1,0,1823,0_covid_coronavirus_deaths_case,"[(covid, 0.01963411001754776), (coronavirus, 0..."
2,1,1608,1_court_supreme court_supreme_pennsylvania,"[(court, 0.025088527265375762), (supreme court..."
3,2,1249,2_poll_joe biden_joe_biden,"[(poll, 0.020120195966791123), (joe biden, 0.0..."
4,3,1107,3_capitol_riot_capitol riot_rioters,"[(capitol, 0.022553649583503952), (riot, 0.020..."


In [79]:
df.dtypes

id                           object
subreddit                    object
author                       object
created_utc          datetime64[ns]
domain                       object
url                          object
title                        object
num_comments                 object
selftext                     object
score                        object
comments                     object
cleaned_title                object
topic                         int64
cleaned_comments             object
comment_sentiment           float64
created_dt_month             object
dtype: object

In [80]:
df = df.dropna()
df['created_utc'] = pd.to_datetime(df['created_utc'])
df["created_dt_month"] = df['created_utc'].dt.strftime('%m-%Y')

titles = df["cleaned_title"].astype(str)
dates = df['created_utc']

topics_over_time = topic_model.topics_over_time(docs=titles, timestamps=dates, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=30)

26it [04:58, 11.47s/it]


In [67]:
len(titles)

87100

In [82]:
tot = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)
tot

In [83]:
tot.write_html("/content/drive/MyDrive/politics_topics_over_time.html")

In [86]:
topics_over_time.to_csv("/content/drive/MyDrive/topics_over_time_politics.csv", index=False)
representative_docs = topic_model.get_representative_docs()

docs_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in representative_docs.items() ])).T
docs_df.index.name = "topic_nr"
docs_df.to_csv("/content/drive/MyDrive/representative_docs_politics.csv")
docs_df.head()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
topic_nr,,,,,,,,,,,,,,,,,,,,,
13,donald trump election odds cut mike pence deba...,kamala harris favorite win debate mike pence o...,karen pence office say side agree mask end debate,goodbye mike pence,let mike pence handle transition,wait mike pence president,mike pence head georgia friday vaccine update ...,mike pence quarantine aid test positive corona...,vice presidential debate harris pence spar ple...,pence face pressure trump thwart electoral col...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,california governor go party violate virus rule,donald trump claim coronavirus death rate,donald trump claim covid deaths day people die,terrify coronavirus surge land biden lap,terrify coronavirus surge land biden lap,biden address nation pandemic fauci say corona...,months inaction congress approve billion covid...,lawmakers set release billion covid plan,democrats prepare pass covid relief republican...,trump sign coronavirus relief government fund ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,government parent migrant children trump admin...,trump aid drive family separation border document,lawyers parent plus migrant kid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,biden abandon promise provide public healthcar...,president elect biden speak plan health care,president biden sign executive order focus hea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303,trump say biden elect listen scientists,trump biden listen scientists elect,banish trump bring biden fauci aim let science...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
df['topic'] = df['cleaned_title'].apply(lambda x: topic_model.find_topics(x)[0][0])
df.head()

,id,subreddit,author,created_utc,domain,url,title,num_comments,selftext,score,comments,cleaned_title,topic,cleaned_comments,comment_sentiment,created_dt_month
0,j767z3,politics,I_Mispelled_Mispell,2020-10-08 03:59:21,buzzfeednews.com,https://www.buzzfeednews.com/article/stephanie...,Kamala Harris's Energy Towards Pence's Interru...,29,,1,I would bet the writer is white....What the fu...,kamala harris energy pence interruptions debat...,14,bet writer white fuck read read like bad parod...,-0.9513,10-2020
1,j76abt,politics,Xeelee1123,2020-10-08 04:03:49,theguardian.com,https://www.theguardian.com/us-news/2020/oct/0...,Trump calls Covid diagnosis 'blessing from God...,29,,1,Hi `Xeelee1123`. Thank you for participating i...,trump call covid diagnosis bless god amid fals...,0,xeelee thank participate politics submission r...,0.9930,10-2020
2,j76cwv,politics,slagholer,2020-10-08 04:08:46,talkingpointsmemo.com,https://talkingpointsmemo.com/news/pence-remin...,Pence Reminds Us All That The Pre-Trump GOP Is...,71,,1,"But you guys, Trump has Jewish grandchildren. ...",pence remind pre trump gop alive kick terminal...,13,guy trump jewish grandchildren possibly suppor...,0.6494,10-2020
3,j76e7j,politics,NotsoPG,2020-10-08 04:11:19,democracynow.org,https://www.democracynow.org/2020/10/7/headlin...,"NYT: Jeff Sessions Defended Child Separations,...",14,,1,He got better at picking the worst people over...,nyt jeff sessions defend child separations say...,82,get better pick worst people time strike gold ...,-0.9914,10-2020
4,j76eku,politics,0belvedere,2020-10-08 04:12:05,nytimes.com,https://www.nytimes.com/2020/10/07/us/politics...,A fly sat atop Mike Pence's head for two minut...,44,,1,well put. thanks....The fly explains its think...,fly sit atop mike pence head minutes debate,13,thank fly explain think https twitter com vinn...,-0.1010,10-2020


In [88]:
df['cleaned_comments'] = df['comments'].map(preprocess)
df.to_csv("/content/drive/MyDrive/reddit_politics_data_with_topics.csv", index=False)
df.head()

,id,subreddit,author,created_utc,domain,url,title,num_comments,selftext,score,comments,cleaned_title,topic,cleaned_comments,comment_sentiment,created_dt_month
0,j767z3,politics,I_Mispelled_Mispell,2020-10-08 03:59:21,buzzfeednews.com,https://www.buzzfeednews.com/article/stephanie...,Kamala Harris's Energy Towards Pence's Interru...,29,,1,I would bet the writer is white....What the fu...,kamala harris energy pence interruptions debat...,14,bet writer white fuck read read like bad parod...,-0.9513,10-2020
1,j76abt,politics,Xeelee1123,2020-10-08 04:03:49,theguardian.com,https://www.theguardian.com/us-news/2020/oct/0...,Trump calls Covid diagnosis 'blessing from God...,29,,1,Hi `Xeelee1123`. Thank you for participating i...,trump call covid diagnosis bless god amid fals...,0,xeelee thank participate politics submission r...,0.9930,10-2020
2,j76cwv,politics,slagholer,2020-10-08 04:08:46,talkingpointsmemo.com,https://talkingpointsmemo.com/news/pence-remin...,Pence Reminds Us All That The Pre-Trump GOP Is...,71,,1,"But you guys, Trump has Jewish grandchildren. ...",pence remind pre trump gop alive kick terminal...,13,guy trump jewish grandchildren possibly suppor...,0.6494,10-2020
3,j76e7j,politics,NotsoPG,2020-10-08 04:11:19,democracynow.org,https://www.democracynow.org/2020/10/7/headlin...,"NYT: Jeff Sessions Defended Child Separations,...",14,,1,He got better at picking the worst people over...,nyt jeff sessions defend child separations say...,82,get better pick worst people time strike gold ...,-0.9914,10-2020
4,j76eku,politics,0belvedere,2020-10-08 04:12:05,nytimes.com,https://www.nytimes.com/2020/10/07/us/politics...,A fly sat atop Mike Pence's head for two minut...,44,,1,well put. thanks....The fly explains its think...,fly sit atop mike pence head minutes debate,13,thank fly explain think https twitter com vinn...,-0.1010,10-2020


In [89]:
sentiment_analyzer = SentimentIntensityAnalyzer()
df["comment_sentiment"] = df['cleaned_comments'].apply(lambda x: sentiment_analyzer.polarity_scores(x)["compound"])

In [90]:
df.head()

,id,subreddit,author,created_utc,domain,url,title,num_comments,selftext,score,comments,cleaned_title,topic,cleaned_comments,comment_sentiment,created_dt_month
0,j767z3,politics,I_Mispelled_Mispell,2020-10-08 03:59:21,buzzfeednews.com,https://www.buzzfeednews.com/article/stephanie...,Kamala Harris's Energy Towards Pence's Interru...,29,,1,I would bet the writer is white....What the fu...,kamala harris energy pence interruptions debat...,14,bet writer white fuck read read like bad parod...,-0.9513,10-2020
1,j76abt,politics,Xeelee1123,2020-10-08 04:03:49,theguardian.com,https://www.theguardian.com/us-news/2020/oct/0...,Trump calls Covid diagnosis 'blessing from God...,29,,1,Hi `Xeelee1123`. Thank you for participating i...,trump call covid diagnosis bless god amid fals...,0,xeelee thank participate politics submission r...,0.9930,10-2020
2,j76cwv,politics,slagholer,2020-10-08 04:08:46,talkingpointsmemo.com,https://talkingpointsmemo.com/news/pence-remin...,Pence Reminds Us All That The Pre-Trump GOP Is...,71,,1,"But you guys, Trump has Jewish grandchildren. ...",pence remind pre trump gop alive kick terminal...,13,guy trump jewish grandchildren possibly suppor...,0.6494,10-2020
3,j76e7j,politics,NotsoPG,2020-10-08 04:11:19,democracynow.org,https://www.democracynow.org/2020/10/7/headlin...,"NYT: Jeff Sessions Defended Child Separations,...",14,,1,He got better at picking the worst people over...,nyt jeff sessions defend child separations say...,82,get better pick worst people time strike gold ...,-0.9914,10-2020
4,j76eku,politics,0belvedere,2020-10-08 04:12:05,nytimes.com,https://www.nytimes.com/2020/10/07/us/politics...,A fly sat atop Mike Pence's head for two minut...,44,,1,well put. thanks....The fly explains its think...,fly sit atop mike pence head minutes debate,13,thank fly explain think https twitter com vinn...,-0.1010,10-2020


In [91]:
# df['created_utc'] = pd.to_datetime(df['created_utc'],errors = "coerce")
df["created_dt_month"] = df['created_utc'].dt.strftime('%m-%Y')

In [92]:
sentiment_df = df[["created_dt_month", "topic", "comment_sentiment"]].groupby(["created_dt_month", "topic"]).mean()
sentiment_df.head()

comment_sentiment
created_dt_month topic                   
01-2020          -1             -0.985000
                  0             -0.512260
                  1              0.087667
                  2              0.226225
                  3             -0.970900

In [93]:
sentiment_df = sentiment_df.reset_index()

In [ ]:
freq

In [101]:
map_topic = freq[["Topic", "Name"]]
map_topic.columns = ['topic', 'name']
merged_df = sentiment_df.merge(map_topic, on=['topic'])

In [102]:
merged_df["created_dt_month"] = pd.to_datetime(merged_df["created_dt_month"])
merged_df = merged_df.sort_values("created_dt_month")
merged_df = merged_df[merged_df['topic']!=-1]
merged_df.head()

,created_dt_month,topic,comment_sentiment,name
1748,2020-01-01,112,-0.346500,112_census_halt census_census bureau_bureau
3864,2020-01-01,357,-0.982800,357_repairman_captain_houston police_houston
1765,2020-01-01,113,0.307167,113_lincoln project_lincoln_project_lincoln pr...
464,2020-01-01,26,-0.965875,26_barr_attorney general_attorney_general
1783,2020-01-01,114,-0.046970,114_steal_steal election_election steal_steal ...


In [103]:
plot_df = merged_df[merged_df['topic'].isin([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])]
# plot_df["created_dt_month"] = pd.to_datetime(plot_df["created_dt_month"])
fig = px.line(plot_df, x="created_dt_month", y="comment_sentiment", color="name")
fig.show()

In [104]:
fig.write_html("/content/drive/MyDrive/politics_comments_sentiments.html")

In [105]:
merged_df.to_csv("/content/drive/MyDrive/politics_comment_sentiment.csv", index=False)

In [106]:
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
docs = titles
# topic_model = BERTopic(verbose=True, n_gram_range=(1, 3),nr_topics='auto',min_topic_size=100)
# topics, _ = topic_model.fit_transform(docs)

# Preprocess Documents
documents = pd.DataFrame({"Document": docs,
                          "ID": range(len(docs)),
                          "Topic": topics})
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)

# Extract vectorizer and analyzer from BERTopic
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
coherence = coherence_model.get_coherence()
print(coherence)

0.25925442012526834
